In [55]:
import datetime
import calendar
import numpy as np
import pandas as pd

from pyomo.environ import *
from pyomo.opt import SolverFactory
from datetime import date
from consumption import get_monthly_num_days_pct, annual_consumption

In [31]:
operating_hours=[8, 8, 8, 8, 8, 0, 0]

days_off = [operating_hours[i] > 0 for i in range(len(operating_hours))]
num_days_off = len([i for i in range(len(operating_hours)) if operating_hours[i] > 0])

print days_off
print num_days_off
print num_days_off/7.0
#len([operating_hours[i] > 0 for i in range(len(operating_hours))])

[True, True, True, True, True, False, False]
5
0.714285714286


In [44]:
year = None
if year == None or year < 2000:
    print "None"
else:
    print "OK"

np.zeros(7)
datetime.date(2018, 4, 2).weekday()

temp_num_weekdays = [12]
temp_num_weekdays[0] = np.zeros(7)
temp_num_weekdays[0][0] = 3

temp_num_weekdays

None


[array([3., 0., 0., 0., 0., 0., 0.])]

In [53]:
operating_hours=[8., 8., 8., 8., 8., 0., 0.]
weekdays = [1, 2, 1, 2, 1, 2, 1]

[a*b for a,b in zip(operating_hours, weekdays)]

[1 if operating_hours[i] > 0 else 0 for i in range(len(operating_hours))]

[1, 1, 1, 1, 1, 0, 0]

In [64]:
daily_df = pd.DataFrame()
daily_df['num_hours'] = np.zeros(365)
daily_df['num_hours_pct'] = np.zeros(365)
daily_df['date'] = datetime.datetime.today()

daily_df.loc[1, 'num_hours'] = 9

daily_df.loc[2, 'date']

Timestamp('2018-04-02 17:50:58.953342')

In [4]:
# Create a solver based on Gnu Linear Programming Kit (glpk)
opt = SolverFactory("glpk")
infinity = float('inf')

# Create an abstract model
model = AbstractModel()

In [5]:
##############################################
# Sets
##############################################
# http://pyomo.readthedocs.io/en/latest/data/datfiles.html

# Months
model.m = Set()

# Hours
model.h = Set()

# Days of Week
model.dw = Set()

# Date
model.d = Set()

# Indices for consumption
model.cons_ind = Set(model.m, model.h, model.dw)

In [6]:
##############################################
# Variables
##############################################

# Recommended system capacity
model.x = Var(model.m, within=NonNegativeReals)

In [7]:
##############################################
# Params
##############################################

# model.n = Param(default=4)
# # Cost of each food
# model.c    = Param(model.F, within=PositiveReals)
# # Amount of nutrient in each food
# model.a    = Param(model.F, model.N, within=NonNegativeReals)
# # Lower and upper bound on each nutrient
# model.Nmin = Param(model.N, within=NonNegativeReals, default=0.0)
# model.Nmax = Param(model.N, within=NonNegativeReals, default=infinity)
# # Volume per serving of food
# model.V    = Param(model.F, within=PositiveReals)
# # Maximum volume of food consumed
# model.Vmax = Param(within=PositiveReals)

# Costs
# param fixed_pc := 500
# param var_pc_solar_mod := 1
# param var_pc_inverter := 0.25
# param var_pc_electrical := 0.25
# param var_pc_structural := 0.1

model.fixed_pc = Param(default=500)
model.var_pc_solar_mod = Param(default=1) # per Watt
model.var_pc_inverter = Param(default=0.25)  # per Watt
model.var_pc_electrical = Param(default=0.25) # per Watt
model.var_pc_structural = Param(default=0.1) # per Watt
model.tou = Param(default=0.000162) # per Watt
model.nem = Param(default=0.000140) # per Watt

# Consumption(months, hours)
model.cons = Param(model.m, model.h, model.dw, within=NonNegativeReals, default=0.0)

# Irradiance(months, hours): used to calculate generation
model.spi = Param(model.m, model.h, within=NonNegativeReals, default=0.0)
# Minimum irradiance
model.min_spi = Param(model.m, model.h, within=NonNegativeReals, default=0.0)
# Maximum irradiance
model.max_spi = Param(model.m, model.h, within=NonNegativeReals, default=1.0)

# Minimum system capacity
model.min_syscap = Param(model.m, within=NonNegativeReals, default=1.0)
# Maximum system capacity
model.max_syscap = Param(model.m, within=NonNegativeReals, default=infinity)

In [8]:
##############################################
# Rules
############################################## 

# # def o_rule(model):
# #     return summation(model.x)

# def energy_cost_savings(model):
#     return 0

# def solar_project_costs(model):
#     return 0

# # ROI = energy cost savings /  solar project costs
# def roi_rule(model):
#     return energy_cost_savings(model) / solar_project_costs(model)

# # def npv(model):
# #     return model

# # Objective function rules: Maximize ROI
# model.cost = Objective(rule = roi_rule)

In [9]:
##############################################
# Constraints
##############################################

# model.c = ConstraintList()

## Generate Sets

In [10]:
# Generate .tab files

num_months = 12
num_week_days = 7
num_month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
num_hours = 24
daily_cons = 3600 #KWh
hourly_cons_pct = [0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 
                  0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417]
hourly_spi = [0, 0, 0, 0, 0, 0, 0, 0.1, 0.2, 0.4, 0.5, 0.65, 
              0.80, 0.85, 0.92, 0.87, 0.80, 0.70, 0.60, 0.40, 0.20, 0, 0, 0]

# Generate .tab file for model.m
f = open("tab/m.tab","w+")
f.write("m")
for i in range(num_months):
    f.write("\n%d" % (i+1))
f.close()

# Generate .tab file for model.h
f = open("tab/h.tab","w+")
f.write("h")
for i in range(num_hours):
    f.write("\n%d" % i)
f.close()

# Generate .tab file for model.dw
f = open("tab/dw.tab","w+")
f.write("dw")
for i in range(num_week_days):
    f.write("\n%d" % (i+1))
f.close()

# Generate .tab file for model.d
f = open("tab/d.tab","w+")
f.write("d")
for i in range(num_months):
    for j in range(num_month_days[i]):
        f.write("\nm%d_d%d" % (i+1, j+1))
f.close()

# Generate .tab file for model.cons
f = open("tab/cons.tab","w+")
f.write("m\th\tdw\tcons")
for i in range(num_months):
    for j in range(num_week_days):
        for k in range(num_hours):
            f.write("\n%d\t%d\t%d\t%f" % (i+1, j+1, k, daily_cons * hourly_cons_pct[k]))
f.close()

# Generate .tab file for model.spi
f = open("tab/spi.tab","w+")
f.write("m\th\tspi")
for i in range(num_months):
    for j in range(num_hours):
        f.write("\n%d\t%d\t%f" % (i+1, j, hourly_spi[j]))
f.close()

In [11]:
# Create DataPortal object
data = DataPortal()

# Load sets
data.load(filename="tab/m.tab", set=model.m)
data.load(filename="tab/h.tab", set=model.h)
data.load(filename="tab/d.tab", set=model.d)
data.load(filename="tab/dw.tab", set=model.dw)

print(data['m'])
print(data['h'])
print(data['d'])
print(data['dw'])

# Load parameters
data.load(filename="tab/cons.tab", param=model.cons)
data.load(filename="tab/spi.tab", param=model.spi)

print(data['cons'])
print(data['spi'])

# instance = model.create(data)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
['m1_d1', 'm1_d2', 'm1_d3', 'm1_d4', 'm1_d5', 'm1_d6', 'm1_d7', 'm1_d8', 'm1_d9', 'm1_d10', 'm1_d11', 'm1_d12', 'm1_d13', 'm1_d14', 'm1_d15', 'm1_d16', 'm1_d17', 'm1_d18', 'm1_d19', 'm1_d20', 'm1_d21', 'm1_d22', 'm1_d23', 'm1_d24', 'm1_d25', 'm1_d26', 'm1_d27', 'm1_d28', 'm1_d29', 'm1_d30', 'm1_d31', 'm2_d1', 'm2_d2', 'm2_d3', 'm2_d4', 'm2_d5', 'm2_d6', 'm2_d7', 'm2_d8', 'm2_d9', 'm2_d10', 'm2_d11', 'm2_d12', 'm2_d13', 'm2_d14', 'm2_d15', 'm2_d16', 'm2_d17', 'm2_d18', 'm2_d19', 'm2_d20', 'm2_d21', 'm2_d22', 'm2_d23', 'm2_d24', 'm2_d25', 'm2_d26', 'm2_d27', 'm2_d28', 'm3_d1', 'm3_d2', 'm3_d3', 'm3_d4', 'm3_d5', 'm3_d6', 'm3_d7', 'm3_d8', 'm3_d9', 'm3_d10', 'm3_d11', 'm3_d12', 'm3_d13', 'm3_d14', 'm3_d15', 'm3_d16', 'm3_d17', 'm3_d18', 'm3_d19', 'm3_d20', 'm3_d21', 'm3_d22', 'm3_d23', 'm3_d24', 'm3_d25', 'm3_d26', 'm3_d27', 'm3_d28', 'm3_d29', 'm3_d30', 'm3_d31',

## Backups

In [59]:
#!cat optimization.dat

In [7]:
print date.today().weekday() + 1
print calendar.day_name[date.today().weekday()]

3
Wednesday


In [ ]:
##############################################
# Variables
##############################################

model.x = Var(RangeSet(model.n), within=Binary)

In [3]:
##############################################
# Rules
############################################## 

# def o_rule(model):
#     return summation(model.x)

def energy_cost_savings(model):
    return 0

def solar_project_costs(model):
    return 0

# ROI = energy cost savings /  solar project costs
def roi_rule(model):
    return energy_cost_savings(model) / solar_project_costs(model)

# def npv(model):
#     return model

# Objective function rules: Maximize ROI
model.cost = Objective(rule = roi_rule)

In [4]:
##############################################
# Constraints
##############################################

model.c = ConstraintList()

In [6]:
##############################################
# Examine solutions
##############################################

# Iterate to eliminate the previously found solution
for i in range(5):
    instance.solutions.load_from(results)
    expr = 0
    for j in instance.x:
        if instance.x[j].value == 0:
            expr += instance.x[j]
        else:
            expr += (1-instance.x[j])
    instance.c.add( expr >= 1 )
    results = opt.solve(instance)
    print ("\n===== iteration",i)
    instance.display()

('\n===== iteration', 0)
Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   1.0

  Constraints:
    c : Size=1
        Key : Lower : Body : Upper
          1 :   1.0 :  1.0 :  None
('\n===== iteration', 1)
Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objec

In [5]:
##############################################
# Optimize
##############################################

# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance)
instance.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    c : Size=0
        Key : Lower : Body : Upper
